# 4. Hyperparameter search for pretraining phase

In [1]:
# allows for import from `src` directory
import sys 
sys.path.append('../')

from src import data
from src import utils

from models.encoder_decoder_dropout import *

## 4.1 Set up data

In [2]:
# fixed data parameters
data_params = {
    'n_input_steps': 48,
    'n_output_steps': 12
}

# run the data preprocessing pipeline to create dataset
_, _, samples = data.pipeline(data_params['n_input_steps'], data_params['n_output_steps'])
datasets = data.get_datasets(samples, data_params['n_input_steps'])


Data already downloaded
43910 train rows from 2012-10-02 09:00:00 to 2017-10-05 23:00:00
4320 valid rows from 2017-10-05 23:00:00 to 2018-04-03 23:00:00
4321 test rows from 2018-04-03 23:00:00 to 2018-09-30 23:00:00
16625 samples of 48 input steps and 12 output steps in train
3534 samples of 48 input steps and 12 output steps in valid
4020 samples of 48 input steps and 12 output steps in test


## 4.2 Bayesian optimisation

### Get device

GPU is recommended as this is a compute heavy job

In [3]:
utils.get_device()

device(type='cuda', index=0)

### Define function to optimise

Wrap the `utils.train` and `utils.evaluate` to define the function to be optimised.

This function takes in a dictionary of hyperparameters and returns the loss value on the validation data using the model trained according to the passed in hyperparameters. We want to know which hyperparameters will minimise this validation loss, so we will use `Ax` to do a guided search using Bayesian optimisation.

In [4]:
def train_evaluate(params):
    device = utils.get_device()
    dataloaders = data.get_dataloaders(datasets, params.get('batch_size'))
    in_features = dataloaders['train'].dataset.X.shape[-1]
    model = VDEncoderDecoder(in_features=in_features, 
                             output_steps=params.get('n_output_steps', 12),
                             p=params.get('variational_dropout_p')
                            ).to(device)
    model,_ = utils.train(device=device, model=model, dataloader=dataloaders['train'], params=params, use_tqdm=True)
    return utils.evaluate(device=device, model=model, valid_loader=dataloaders['valid'])

### Set up `Ax` client

We mainly follow the tutorial at https://ax.dev/tutorials/gpei_hartmann_service.html

In [5]:
from ax.service.ax_client import AxClient

ax_client = AxClient(enforce_sequential_optimization=False)

# define hyperparameter bounds 
ax_client.create_experiment(
    name='pretraining',
    parameters=[
        {"name": "num_epochs", "type": "range", "bounds": [150, 200]},
        {"name": "learning_rate", "type": "range", "bounds": [5e-4, 1e-3], "log_scale": True},
        {"name": "batch_size", "type": "range", "bounds": [64, 1024]},
        {"name": "variational_dropout_p", "type": "range", "bounds": [0.2,0.5]}
    ],
    objective_name='loss',
    minimize=True
)

def evaluate(parameters):
    return {"pretraining": train_evaluate(parameters)}


[INFO 09-09 20:34:46] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 2 decimal points.
[INFO 09-09 20:34:46] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation strategy: GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 5 trials, GPEI for subsequent trials]). Iterations after 5 will take longer to generate due to  model-fitting.


### Launch the optimisation loop

In [ ]:
# run 30 jobs
for i in range(30):
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=evaluate(parameters)['pretraining']['loss'])


[INFO 09-09 20:34:58] ax.service.ax_client: Generated new trial 0 with parameters {'num_epochs': 189, 'learning_rate': 0.0, 'batch_size': 586, 'variational_dropout_p': 0.3}.
Epoch=188 | [16625|16625]	loss=1.2536: 100%|██████████| 189/189 [03:30<00:00,  1.11s/it]
[INFO 09-09 20:38:31] ax.service.ax_client: Completed trial 0 with data: {'loss': (1.23, None)}.
[INFO 09-09 20:38:31] ax.service.ax_client: Generated new trial 1 with parameters {'num_epochs': 195, 'learning_rate': 0.0, 'batch_size': 999, 'variational_dropout_p': 0.49}.
Epoch=194 | [16625|16625]	loss=1.4263: 100%|██████████| 195/195 [03:06<00:00,  1.04it/s]
[INFO 09-09 20:41:38] ax.service.ax_client: Completed trial 1 with data: {'loss': (0.69, None)}.
[INFO 09-09 20:41:38] ax.service.ax_client: Generated new trial 2 with parameters {'num_epochs': 152, 'learning_rate': 0.0, 'batch_size': 186, 'variational_dropout_p': 0.41}.
Epoch=151 | [16625|16625]	loss=1.3494: 100%|██████████| 152/152 [05:25<00:00,  2.14s/it]
[INFO 09-09 20:

### Get best parameters

In [7]:
ax_client.get_best_parameters()

({'num_epochs': 200,
  'learning_rate': 0.0009002389954280866,
  'batch_size': 64,
  'variational_dropout_p': 0.2000000000364784},
 ({'loss': 0.19481891684816835}, {'loss': {'loss': 0.00042067191742800174}}))

### Save results

In [8]:
ax_client.save_to_json_file()

[INFO 09-09 23:22:21] ax.service.ax_client: Saved JSON-serialized state of optimization to `ax_client_snapshot.json`.
